# 🌳 Advanced Tree Structures for ML

**Welcome, St. Mark!** In this notebook, we'll explore advanced tree structures that power efficient ML systems.

We'll implement:
1. **Balanced Binary Search Trees** - Self-balancing BSTs for ordered data
2. **B-Trees** - Multiway trees for disk-based storage
3. **Heaps** - Priority queues for efficient ordering

By the end, you'll understand how tree structures enable scalable ML data management.

In [ ]:
import heapq
import matplotlib.pyplot as plt
import numpy as np
import time

print("Setting up advanced tree structures for ML...")

## Method 1: Balanced Binary Search Trees - AVL Trees

**Healthcare Analogy:** Like organizing patient records by ID - always balanced for fast access.

**ML Application:** Ordered data structures, database indexing, sorted collections.

In [ ]:
class AVLNode:
    def __init__(self, key, value=None):
        self.key = key
        self.value = value
        self.left = None
        self.right = None
        self.height = 1

class AVLTree:
    def __init__(self):
        self.root = None
        
    def _height(self, node):
        return node.height if node else 0
    
    def _balance_factor(self, node):
        return self._height(node.left) - self._height(node.right) if node else 0
    
    def _update_height(self, node):
        node.height = 1 + max(self._height(node.left), self._height(node.right))
    
    def _rotate_right(self, y):
        x = y.left
        T2 = x.right
        x.right = y
        y.left = T2
        self._update_height(y)
        self._update_height(x)
        return x
    
    def _rotate_left(self, x):
        y = x.right
        T2 = y.left
        y.left = x
        x.right = T2
        self._update_height(x)
        self._update_height(y)
        return y
    
    def _balance(self, node):
        balance = self._balance_factor(node)
        
        if balance > 1:
            if self._balance_factor(node.left) < 0:
                node.left = self._rotate_left(node.left)
            return self._rotate_right(node)
        
        if balance < -1:
            if self._balance_factor(node.right) > 0:
                node.right = self._rotate_right(node.right)
            return self._rotate_left(node)
        
        return node
    
    def insert(self, key, value=None):
        self.root = self._insert_recursive(self.root, key, value)
    
    def _insert_recursive(self, node, key, value):
        if not node:
            return AVLNode(key, value)
        
        if key < node.key:
            node.left = self._insert_recursive(node.left, key, value)
        elif key > node.key:
            node.right = self._insert_recursive(node.right, key, value)
        else:
            node.value = value
            return node
        
        self._update_height(node)
        return self._balance(node)
    
    def search(self, key):
        return self._search_recursive(self.root, key)
    
    def _search_recursive(self, node, key):
        if not node or node.key == key:
            return node.value if node else None
        
        if key < node.key:
            return self._search_recursive(node.left, key)
        else:
            return self._search_recursive(node.right, key)

In [ ]:
# Healthcare application: Patient record indexing
patient_records = {
    "PAT001": {"name": "Adebayo Johnson", "age": 45, "condition": "Hypertension"},
    "PAT002": {"name": "Fatima Abubakar", "age": 32, "condition": "Diabetes"},
    "PAT003": {"name": "Chukwuma Nwosu", "age": 28, "condition": "Malaria"}
}

patient_index = AVLTree()
for patient_id, record in patient_records.items():
    patient_index.insert(patient_id, record)

print("🏥 Patient Record AVL Tree Index:")
print(f"Indexed {len(patient_records)} patient records")

for patient_id in patient_records.keys():
    record = patient_index.search(patient_id)
    if record:
        print(f"✓ Found {patient_id}: {record['name']} - {record['condition']}")

**Healthcare Analysis:**

- **Patient Indexing:** Fast lookup by patient ID with ordered access
- **Automatic Balancing:** Maintains optimal performance as data grows

**Nigerian Healthcare Impact:** Enable efficient patient record management across multiple hospitals.

## Method 2: B-Trees - Multiway Trees for Disk Storage

**Healthcare Analogy:** Like organizing medical records in a filing cabinet with multiple folders per drawer.

**ML Application:** Database indexing, file system organization, large-scale data storage.

In [ ]:
class BTreeNode:
    def __init__(self, leaf=False):
        self.leaf = leaf
        self.keys = []
        self.children = []
        self.values = []

class BTree:
    def __init__(self, degree=2):
        self.root = BTreeNode(leaf=True)
        self.degree = degree
        
    def _split_child(self, parent, i):
        degree = self.degree
        child = parent.children[i]
        
        new_node = BTreeNode(leaf=child.leaf)
        
        mid_key = child.keys[degree - 1]
        new_node.keys = child.keys[degree:(2*degree - 1)]
        child.keys = child.keys[0:(degree - 1)]
        
        if not child.leaf:
            new_node.children = child.children[degree:(2*degree)]
            child.children = child.children[0:degree]
        else:
            new_node.values = child.values[degree:(2*degree - 1)]
            child.values = child.values[0:(degree - 1)]
        
        parent.children.insert(i + 1, new_node)
        parent.keys.insert(i, mid_key)
    
    def insert(self, key, value=None):
        root = self.root
        
        if len(root.keys) == (2 * self.degree - 1):
            new_root = BTreeNode()
            self.root = new_root
            new_root.children.append(root)
            self._split_child(new_root, 0)
            self._insert_non_full(new_root, key, value)
        else:
            self._insert_non_full(root, key, value)
    
    def _insert_non_full(self, node, key, value):
        i = len(node.keys) - 1
        
        if node.leaf:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            node.keys.insert(i + 1, key)
            node.values.insert(i + 1, value)
        else:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            
            if len(node.children[i].keys) == (2 * self.degree - 1):
                self._split_child(node, i)
                if key > node.keys[i]:
                    i += 1
            
            self._insert_non_full(node.children[i], key, value)
    
    def search(self, key):
        return self._search_recursive(self.root, key)
    
    def _search_recursive(self, node, key):
        i = 0
        while i < len(node.keys) and key > node.keys[i]:
            i += 1
        
        if i < len(node.keys) and key == node.keys[i]:
            return node.values[i] if node.leaf else None
        
        if node.leaf:
            return None
        
        return self._search_recursive(node.children[i], key)

In [ ]:
# Healthcare application: Large-scale medical database indexing
medical_database = {
    "ICD-A00": "Cholera",
    "ICD-I10": "Essential hypertension",
    "ICD-E46": "Unspecified protein-calorie malnutrition"
}

diagnosis_index = BTree(degree=2)
for code, description in medical_database.items():
    diagnosis_index.insert(code, description)

print("🏥 Medical Diagnosis B-Tree Index:")
print(f"Indexed {len(medical_database)} ICD diagnosis codes")

for code in medical_database.keys():
    description = diagnosis_index.search(code)
    if description:
        print(f"✓ {code}: {description}")

**Healthcare Analysis:**

- **ICD Code Indexing:** Fast lookup of medical diagnosis codes
- **Disk Efficiency:** Minimizes I/O operations for large databases

**Nigerian Healthcare Impact:** Enable efficient medical coding and diagnosis lookup across national health systems.

## Method 3: Heaps - Priority Queues for ML

**Healthcare Analogy:** Like a hospital triage system - most urgent patients treated first.

**ML Application:** Priority queues, beam search, top-k selection, scheduling.

In [ ]:
class PriorityQueue:
    def __init__(self, max_heap=False):
        self.heap = []
        self.max_heap = max_heap
        
    def push(self, item, priority):
        heapq.heappush(self.heap, (priority if not self.max_heap else -priority, item))
    
    def pop(self):
        if self.heap:
            priority, item = heapq.heappop(self.heap)
            return item, (priority if not self.max_heap else -priority)
        return None, None
    
    def size(self):
        return len(self.heap)
    
    def is_empty(self):
        return len(self.heap) == 0

In [ ]:
# Healthcare application: Hospital triage system
triage_queue = PriorityQueue(max_heap=True)

patients = [
    {"name": "Ahmadu Bello", "condition": "Heart attack", "priority": 10},
    {"name": "Ngozi Okoye", "condition": "Broken arm", "priority": 3},
    {"name": "Emeka Nwosu", "condition": "Severe infection", "priority": 8}
]

for patient in patients:
    triage_queue.push(patient, patient["priority"])

print("🏥 Hospital Triage Priority Queue:")
print(f"Processing {triage_queue.size()} patients in priority order")

while not triage_queue.is_empty():
    patient, priority = triage_queue.pop()
    print(f"{patient['name']} - {patient['condition']} (Priority: {priority})")

**Healthcare Analysis:**

- **Triage System:** Patients treated by medical urgency priority
- **Resource Allocation:** Critical cases get immediate attention

**Nigerian Healthcare Impact:** Enable proper patient prioritization in overcrowded hospital systems.

## 🎯 Key Takeaways and Nigerian Healthcare Applications

**Algorithm Summary:**
- **AVL Trees:** Self-balancing BSTs for ordered data with guaranteed O(log n) performance
- **B-Trees:** Multiway trees optimized for disk-based storage and large datasets
- **Heaps:** Priority queues for efficient access to minimum/maximum elements

**Healthcare Translation - Mark:**
Imagine building Nigeria's healthcare data infrastructure:
- **AVL Trees:** Patient records indexed by ID with fast ordered access
- **B-Trees:** National health database with millions of records
- **Heaps:** Hospital triage systems prioritizing critical patients

**Performance achieved:** Tree structures enable efficient data management for large-scale ML healthcare systems!